In [ ]:
import cn_clip.clip as clip
from cn_clip.clip import load_from_name, available_models
from PIL import Image
import torch

In [ ]:
available_models()

In [ ]:
model, preprocess = load_from_name("ViT-B-16", device="mps", download_root="./")
model.eval()

In [ ]:
image = preprocess(Image.open("3F97341B-5872-4E07-A1AC-70871839F8BA_1_105_c.jpeg")).unsqueeze(0).to("mps")

In [ ]:
# 读取 ImageNet 的标签
import json
text_labels = []
with open("label_cn.txt") as f:
    text_labels = f.readlines()
# remove whitespace characters like `\n` at the end of each line
text_labels = [x.strip() for x in text_labels]
labels = clip.tokenize(text_labels).to("mps")

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_feature = model.encode_text(labels)
    text_feature /= text_feature.norm(dim=-1, keepdim=True)

similarity = (100.0 * image_features @ text_feature.T).softmax(dim=-1)
# find top 5 labels with highest similarity
values, indices = similarity[0].topk(5)
# print("Label and confidence:")
for value, index in zip(values, indices):
    print(f"{text_labels[index]}: {100 * value.item():.2f}%")